In [46]:
import pandas as pd
import numpy as np

In [18]:
train = pd.read_table('train_c.csv', sep=';', encoding='utf-8')

In [19]:
train.shape

(36, 2)

In [20]:
train.head()

,i,text
0,1,"Он вынул связку ключей, отделил один, отпер дв..."
1,1,"Крушит, рушит. Ключ от квартиры взяли у управд..."
2,1,"Ключ от этой комнаты был один на всех, хранилс..."
3,1,"Взяла ключ, спустилась в подвал, отомкнула ком..."
4,1,"Клёкотов уже и ключом в замке ковыряет, и войд..."


In [22]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [25]:
from string import punctuation
punct = punctuation+'«»—…“”*№–'

In [23]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words]
    
    return words

In [26]:
train['lemmas'] = train['text'].apply(normalize)

In [27]:
train.head()

,i,text,lemmas
0,1,"Он вынул связку ключей, отделил один, отпер дв...","[он, вынуть, связка, ключ, отделить, один, отп..."
1,1,"Крушит, рушит. Ключ от квартиры взяли у управд...","[крушить, рушить, ключ, от, квартира, взять, у..."
2,1,"Ключ от этой комнаты был один на всех, хранилс...","[ключ, от, этот, комната, быть, один, на, весь..."
3,1,"Взяла ключ, спустилась в подвал, отомкнула ком...","[взять, ключ, спуститься, в, подвал, отомкнуть..."
4,1,"Клёкотов уже и ключом в замке ковыряет, и войд...","[клёкот, уже, и, ключ, в, замок, ковырять, и, ..."


In [29]:
from collections import Counter

In [51]:
its0 = train['lemmas'][train['i'] == 0]
its1 = train['lemmas'][train['i'] == 1]

In [60]:
arr0 = []
for i in its0:
    arr0 += i
    
arr1 = []
for i in its1:
    arr1 += i

In [63]:
cntr0 = Counter(arr0)
cntr1 = Counter(arr1)

In [74]:
dict0 = {k:v for k,v in cntr0.items() if v > 1}

In [77]:
dict1 = {k:v for k,v in cntr1.items() if v > 1}

In [78]:
dict1

{'он': 9,
 'связка': 3,
 'ключ': 21,
 'один': 2,
 'дверь': 6,
 'войти': 4,
 'от': 4,
 'квартира': 6,
 'взять': 3,
 'у': 3,
 'в': 12,
 '': 11,
 'этот': 2,
 'комната': 5,
 'быть': 4,
 'на': 6,
 'весь': 4,
 'сестра-хозяйка': 4,
 'и': 23,
 'это': 2,
 'если': 3,
 'значит': 3,
 'нет': 2,
 'ни': 2,
 'свежий': 2,
 'подвал': 2,
 'будто': 2,
 'уже': 4,
 'с': 5,
 'подойти': 2,
 'вот': 7,
 'к': 5,
 'её': 2,
 'надзиратель': 2,
 'выйти': 2,
 'скважина': 2,
 'что': 2,
 'телевизор': 2,
 'по': 2,
 'не': 5,
 'нога': 2,
 'свой': 2,
 'при': 2}

2.

In [82]:
sin_ents = list(cntr0) + list(cntr1)

In [88]:
with open('singles.txt', 'w') as f:
    f.write('\n'.join(sin_ents))

Первый раз

In [89]:
from sklearn.naive_bayes import MultinomialNB

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [91]:
vectorizer = TfidfVectorizer()

In [134]:
train['str'] = [' '.join(map(str, l)) for l in train['lemmas']]

In [155]:
X = train.iloc[:,3]
y = train.iloc[:,0]

In [150]:
X = vectorizer.fit_transform(X)

In [151]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [152]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [153]:
y_predicted = model.predict(X_test)

In [154]:
roc_auc_score(y_test, y_predicted)

1.0

Второй раз (без стоп-слов)

In [105]:
from nltk.corpus import stopwords
stops = set(stopwords.words('russian'))

In [156]:
vectorizer1 = TfidfVectorizer(stop_words=stops)

In [157]:
X1 = vectorizer1.fit_transform(X)

In [158]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.2, random_state=0)

In [159]:
model1 = MultinomialNB()
model1.fit(X1_train, y1_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [160]:
y1_predicted = model1.predict(X1_test)

In [161]:
roc_auc_score(y1_test, y1_predicted)

0.875

Третий раз (без стоп-слов + нечастотных)

In [123]:
stops_exp = set(stopwords.words('russian') + sin_ents)

In [126]:
vectorizer2 = TfidfVectorizer(stop_words=stops_exp)

In [144]:
X2 = vectorizer2.fit_transform(X)

In [145]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, test_size=0.2, random_state=0)

In [146]:
model2 = MultinomialNB()
model2.fit(X2_train, y2_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [147]:
y2_predicted = model2.predict(X2_test)

In [148]:
roc_auc_score(y2_test, y2_predicted)

0.625

In [163]:
with open('stops.txt', 'w') as f:
    f.write('\n'.join(stops_exp))

In [170]:
with open('freq_d_0.txt', 'w') as f:
    for key, value in dict0.items():
        f.write('%s:%s\n' % (key, value))

In [171]:
with open('freq_d_1.txt', 'w') as f:
    for key, value in dict1.items():
        f.write('%s:%s\n' % (key, value))